# Buscador de accesos

## Importación de librerías

In [119]:
import openpyxl

## Lectura de datos

Abrimos libro "Orgánica" y entramos a la hoja activa, "Informe".

In [2]:
wb_o = openpyxl.load_workbook("organica.xlsx")
ws_o = wb_o.active

Creamos un diccionario desde los headers hacia sus respectivas columnas.

In [23]:
organica_c = {}
for cell in ws_o[1]:
  if cell.value != None:
    organica_c[cell.value] = cell.column_letter

Creamos un diccionario desde el RUT de cada persona hacia su respectiva concatenación de "UR" y "Cargo".

In [51]:
rut_dict = {}
for i in range(2,ws_o.max_row):
  rut = ws_o[f"{organica_c['RUT']}{i}"]
  ur = ws_o[f"{organica_c['UR']}{i}"]
  cargo = ws_o[f"{organica_c['Cargo']}{i}"]
  rut_dict[rut.value] = f"{ur.value}-{cargo.value}"

Abrimos el catálogo y guardamos sus respectivas hojas en variables distintas.

In [31]:
wb_c = openpyxl.load_workbook("catalogo.xlsx")
ws_c_1 = wb_c['Sucursales']
ws_c_2 = wb_c['Serv.Centrales']
ws_c_3 = wb_c['Contac Center']

Creamos un diccionario desde los headers hacia sus respectivas columnas.

In [32]:
catalogo_c = {}
for cell in ws_c_1[1]:
  if cell.value != None:
    catalogo_c[cell.value] = cell.column_letter

Función que para cada hoja del catálogo guarda los resultados de cada concatenación de la forma:


```
# Concatenación: {Rol 1: [{Aplicacion: App1, Perfil: Perfil1},
                          ...,
                          {Aplicacion: Appn, Perfil: Perfiln}],
                  ...,
                  Rol n: [{Aplicacion: App1, Perfil: Perfil1},
                          ...,
                          {Aplicacion: Appn, Perfil: Perfiln}]}

```



In [90]:
def make_concat_dict(concat_dict, ws):
  for i in range(2,ws.max_row):
    ur = ws[f"{catalogo_c['UR']}{i}"]
    cargo = ws[f"{catalogo_c['Cargo']}{i}"]
    rol = ws[f"{catalogo_c['Rol']}{i}"]
    app = ws[f"{catalogo_c['Aplicacion']}{i}"]
    perfil = ws[f"{catalogo_c['Perfil']}{i}"]

    concat = f"{ur.value}-{cargo.value}"
    if concat not in concat_dict.keys():
      concat_dict[concat] = {rol.value: [{'Aplicacion': app.value, 'Perfil': perfil.value}]}
    else:
      if rol.value in concat_dict[concat].keys():
        concat_dict[concat][rol.value].append({'Aplicacion': app.value, 'Perfil': perfil.value})
      else:
        concat_dict[concat][rol.value] = [{'Aplicacion': app.value, 'Perfil': perfil.value}]

  return concat_dict

Se corre la función para todas las hojas, guardando el diccionario final como "catalogo_dict".

In [91]:
catalogo_dict = make_concat_dict(make_concat_dict(make_concat_dict({}, ws_c_1),ws_c_2),ws_c_3)

## Búsqueda de accesos

Se define una función que busca todos los accesos según un RUT dado.

In [108]:
def busqueda_de_accesos(rut, rut_dict, catalogo_dict):
  roles = catalogo_dict[rut_dict[rut]]
  for rol in roles:
    print(f'Accesos del Rol: {rol}')
    accesos = roles[rol]
    for id,val in enumerate(accesos):
      print(f"{id+1}. Aplicación: {val['Aplicacion']}, Perfil: {val['Perfil']}")

Ejemplo de esta función:

In [111]:
busqueda_de_accesos('10000005-9', rut_dict, catalogo_dict)

Accesos del Rol: Rol 21
1. Aplicación: App 10, Perfil: Perfil 3
2. Aplicación: App 11, Perfil: Perfil 4
3. Aplicación: App 12, Perfil: Perfil 5
4. Aplicación: App 13, Perfil: Perfil 6
